<a href="https://colab.research.google.com/github/saransh2320/Twiiter-Customer-Support/blob/master/Customer_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import re
import random
import time

In [2]:
# for language Detection
!pip install pycld2

     |████████████████████████████████| 41.4MB 73kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833536 sha256=6a980d2dafda9d19e512178715ae2bfeb33e1f7df57c8c6cc2915dc0193f9d7d
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [3]:
!pip install -q kaggle

In [4]:
kaggle_api_key = '{"username":"hsaransh","key":"d751f4719ec73f12f570241faafd19c9"}'
kaggle_api_key = json.loads(kaggle_api_key)

In [5]:
!mkdir -p ~/.kaggle

In [6]:
with open('kaggle.json', 'w') as json_file:
    json.dump(kaggle_api_key, json_file)

In [7]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 91% 153M/169M [00:04<00:00, 23.3MB/s]
100% 169M/169M [00:04<00:00, 40.4MB/s]


In [10]:
!unzip '/content/customer-support-on-twitter.zip'

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [11]:
import numpy as np
import pandas as pd
import string

import keras
import sklearn
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize

from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [12]:
tweets = pd.read_csv('/content/twcs/twcs.csv')

In [13]:
tweets.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [ ]:
tweets.loc[lambda tf: tf['tweet_id'] == 81]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
73,81,ChipotleTweets,False,Mon Oct 30 15:00:02 +0000 2017,Just stop by in costume 10/31 3pm-close. Offic...,"80,82,83,84,85,86,87,88,89,90,91,92,93,94,95,9...",NaN


In [ ]:
tweets.loc[lambda tf: tf['tweet_id'] == 409]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
301,409,McDonalds,False,Tue Oct 31 16:02:50 +0000 2017,So what’ll it be this #Halloween: a trick or a...,"410,411,412,413,414,415,416,417,418,419,420,42...",NaN


In [14]:
tweets.shape

(2811774, 7)

In [15]:
# First_inbound have all the tweets in_response_to_tweet_id is null (i.e. starting of converstion) and tweets.inbound == true (i.e. started by customer)
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound == True]

# Mergeing above with the whole dataset to get que -> ans
inbounds_and_outbounds = pd.merge(first_inbound,tweets,left_on='tweet_id',right_on='in_response_to_tweet_id')

# Filtering inbound_y == false for only company replies
inbounds_and_outbounds = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]

In [ ]:
inbounds_and_outbounds.head()

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
1,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0
3,18,115713,True,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,17,NaN,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0
4,20,115715,True,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",19,NaN,19,sprintcare,False,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...,NaN,20.0


In [16]:
inbounds_and_outbounds.shape

(794299, 14)

In [19]:
# dropping unnecessary Columns
query_and_response = inbounds_and_outbounds[["text_x","text_y"]]
query_and_response.head()

,text_x,text_y
0,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
1,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...
2,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...
3,@115714 y’all lie about your “great” connectio...,@115713 H there! We'd definitely like to work ...
4,"@115714 whenever I contact customer support, t...",@115715 Please send me a private message so th...


In [20]:
query_and_response.shape

(794299, 2)

In [21]:
# Removing @username

query_and_response['text_x'].replace(to_replace="@[A-Za-z0-9]+", value="", regex=True,inplace=True)

query_and_response['text_y'].replace(to_replace="@[A-Za-z0-9]+", value="", regex=True,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [ ]:
query_and_response.head()

,text_x,text_y
0,is the worst customer service,"Can you please send us a private message, so ..."
1,is the worst customer service,I would love the chance to review the account...
2,is the worst customer service,Hello! We never like our customers to feel li...
3,y’all lie about your “great” connection. 5 ba...,H there! We'd definitely like to work with yo...
4,"whenever I contact customer support, they tel...",Please send me a private message so that I ca...


In [18]:
# Testing pycld2 Library
import pycld2 as cld2
k = (cld2.detect("who is here")[2])[0][1]

In [22]:
# removing non english rows
def is_en(txt):
    try:
        return ((cld2.detect(txt)[2])[0][1])=='en'
    except:
        return False

query_and_response = query_and_response[query_and_response['text_x'].apply(is_en)]
query_and_response = query_and_response[query_and_response['text_y'].apply(is_en)]

In [23]:
query_and_response.shape

(754225, 2)

In [24]:
# lower case
query_and_response["text_x"] = query_and_response["text_x"].str.lower()
query_and_response["text_y"] = query_and_response["text_y"].str.lower()


In [25]:
# PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
    return text_nopunct

query_and_response["text_x"] = query_and_response["text_x"].apply(lambda text: remove_punctuation(text))
query_and_response["text_y"] = query_and_response["text_y"].apply(lambda text: remove_punctuation(text))


In [26]:
query_and_response.head(20)

,text_x,text_y
0,is the worst customer service,can you please send us a private message so t...
1,is the worst customer service,i would love the chance to review the account...
2,is the worst customer service,hello we never like our customers to feel lik...
3,y’all lie about your “great” connection 5 bar...,h there wed definitely like to work with you ...
4,whenever i contact customer support they tell...,please send me a private message so that i ca...
5,actually thats a broken link you sent me and i...,the information pertaining to the account ass...
6,yo spectrum your customer service reps are sup...,hello my apologies for any frustrations or in...
7,my picture on spectrum pretty much every day w...,i apologize for the inconvenience i will be g...
8,somebody from please help meeeeee 😩😩😩😩 im hav...,help has arrived we are sorry to see that you...
9,my friend is without internet we need to play...,have your friend message us\nacm


In [27]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [28]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [29]:
import re

In [30]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

In [31]:
query_and_response["text_x"] = query_and_response["text_x"].apply(lambda text: convert_emoticons(text))
query_and_response["text_y"] = query_and_response["text_y"].apply(lambda text: convert_emoticons(text))


In [32]:
#remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [33]:
query_and_response["text_x"] = query_and_response["text_x"].apply(lambda text: remove_urls(text))
query_and_response["text_y"] = query_and_response["text_y"].apply(lambda text: remove_urls(text))

In [35]:
MAX_VOCAB_SIZE = 2**13

MAX_MESSAGE_LEN = 30

EMBEDDING_SIZE = 100

# CONTEXT_SIZE = 100

BATCH_SIZE = 4

DROPOUT = 0.2

LEARNING_RATE = 0.005

UNK = 0
PAD = 1

START = 2

SUB_BATCH_SIZE = 1000

In [36]:
# Count Vectorizer assign a unique number to every word in corpus


count_vec = CountVectorizer(tokenizer=casual_tokenize,max_features=MAX_VOCAB_SIZE-3)
print("Fitting Count Vectorizer on X and Y text data")

count_vec.fit(tqdm(query_and_response['text_x']+query_and_response['text_y']))
analyzer = count_vec.build_analyzer()

Fitting Count Vectorizer on X and Y text data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
count_vec.vocabulary_.items()

dict_items([('is', 3958), ('the', 7132), ('worst', 7899), ('customer', 2039), ('service', 6409), ('can', 1438), ('you', 7977), ('please', 5456), ('send', 6387), ('us', 7561), ('a', 464), ('private', 5603), ('message', 4701), ('so', 6632), ('that', 7129), ('i', 3737), ('gain', 3090), ('further', 3082), ('details', 2235), ('about', 484), ('your', 7983), ('account', 515), ('would', 7902), ('love', 4491), ('chance', 1566), ('to', 7249), ('review', 6105), ('and', 761), ('provide', 5666), ('assistance', 912), ('hello', 3389), ('we', 7742), ('never', 4930), ('like', 4374), ('our', 5168), ('customers', 2041), ('feel', 2850), ('they', 7144), ('are', 862), ('not', 5000), ('valued', 7596), ('y', 7946), ('’', 8020), ('all', 691), ('lie', 4363), ('“', 8021), ('great', 3228), ('”', 8022), ('connection', 1852), ('5', 295), ('bars', 1071), ('lte', 4507), ('still', 6821), ('won', 7870), ('t', 7000), ('load', 4430), ('something', 6655), ('smh', 6612), ('h', 3277), ('there', 7140), ('wed', 7753), ('defin

In [37]:
# Adding +3 so that we can accomodate UNK(= 0),PAD(= 1),START(= 2)
vocab = {k: v+3 for k,v in count_vec.vocabulary_.items()}

In [38]:
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START

In [39]:
# Reverse vocab will use to converting no. to words
reverse_vocab = {v: k for k, v in vocab.items()}

In [44]:
def word_to_number(sentence):
  full_length = [vocab.get(tok,UNK) for tok in analyzer(sentence)]+[PAD]*MAX_MESSAGE_LEN
  return full_length[:MAX_MESSAGE_LEN] 

In [45]:
word_to_number("what are you doing saransh where is my bag")

[7787,
 865,
 7980,
 2397,
 0,
 7798,
 3961,
 4871,
 1033,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [46]:
def number_to_word(numbers):
  return ' '.join(reverse_vocab[idx] for idx in numbers if idx!=PAD).strip()

In [47]:
number_to_word([7787,
 865,
 7980,
 2397,
 0,
 7798,
 3961,
 4871,
 1033,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1])

'what are you doing __unk__ where is my bag'

In [48]:
x = np.vstack(query_and_response['text_x'].apply(word_to_number).values)

In [49]:
y = np.vstack(query_and_response['text_y'].apply(word_to_number).values)

In [52]:
from sklearn.model_selection import train_test_split

In [54]:
train_x ,test_x ,train_y ,test_y = train_test_split(x,y,test_size=0.2,random_state = 42,shuffle=True)

In [58]:
print("----------")
print("Train Shape X and Y")
print(train_x.shape)
print(train_y.shape)
print("----------")
print("Test Shape X and Y")
print(test_x.shape)
print(test_y.shape)

----------
Train Shape X and Y
(603380, 30)
(603380, 30)
----------
Test Shape X and Y
(150845, 30)
(150845, 30)


In [ ]:
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate,TimeDistributed
# from keras.utils import np_utils

In [ ]:
# def create_model():
#   shared_embedding = Embedding(
#       output_dim = EMBEDDING_SIZE,
#       input_dim = MAX_VOCAB_SIZE,
#       input_length = MAX_MESSAGE_LEN,
#       name='embedding',
#   )

#   encoder_input = INTPUT(
#       shape=(MAX_MESSAGE_LEN,),
#       dtype='int32',
#       name='encoder_input',
#   )

#   embedded_input = shared_embedding(encoder_input)
  
#   encoder_rnn = LSTM(
#       CONTEXT_SIZE,
#       name='encoder',
#       dropout = DROPOUT
#   )

#   context = RepeatVector(MAX_MESSAGE_LEN)(encoder_rnn(embedded_input))
  

hel
